In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 5.0MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [6]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [5]:
ls

 ColabDriveGithub.ipynb             Day4_meta.ipynb       Untitled0.ipynb
'Copy of ColabDriveGithub.ipynb'    dw_matrix/
'Copy of Welcome To Colaboratory'   Testy20200212.ipynb


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [11]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(features, model = DecisionTreeRegressor(max_depth=5)):
  X = df[features].values
  y = df['prices_amountmin'].values
  
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [13]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [14]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [22]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [24]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [25]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)


In [26]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_bag size range', 'feat_place of origin', 'feat_safety feature',
       'feat_made in', 'feat_stylenumber', 'feat_gps', 'feat_metal stamp',
       'feat_feature', 'feat_thick, warm, cozy', 'feat_licensed'],
      dtype='object', length=526)

In [32]:
df[False == df['feat_athlete'].isnull()].shape[0] / df.shape[0] * 100

0.0437636761487965

In [34]:
keys_stat ={}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

keys_stat

{'100% adidas authentic': 0.005470459518599562,
 'accessory type': 0.1422319474835886,
 'adidas': 0.005470459518599562,
 'adjustable': 0.34463894967177244,
 'age': 0.6400437636761488,
 'age end': 0.4431072210065646,
 'age gender group': 0.12582056892778995,
 'age group': 27.64770240700219,
 'age range': 0.5470459518599562,
 'age segment': 0.1422319474835886,
 'age start': 0.4431072210065646,
 'airport friendly': 0.005470459518599562,
 'alarm': 0.23522975929978115,
 'amazonbestsellersrank': 0.06017505470459519,
 'animal type': 0.005470459518599562,
 'antiscratch lens coating': 0.005470459518599562,
 'applicable': 0.02188183807439825,
 'arm': 0.6236323851203501,
 'article': 0.005470459518599562,
 'assembled in country of origin': 2.199124726477024,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'assembled product weight': 1.5536105032822756,
 'athlete': 0.0437636761487965,
 'atpv arc rating': 0.005470459518599562,
 'attachment': 0.03282275711159737,
 'audience': 0.2352

In [103]:
{k:v for k,v in keys_stat.items() if v > 10}

{'age group': 27.64770240700219,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'brand': 48.62691466083151,
 'casual & dress shoe style': 12.910284463894966,
 'color': 47.784463894967175,
 'condition': 18.12910284463895,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838,
 'model': 23.681619256017505,
 'shoe category': 24.376367614879648,
 'shoe size': 23.326039387308533,
 'shoe width': 15.74945295404814,
 'size': 21.422319474835884,
 'style': 14.923413566739608}

In [0]:
df[get_name_feat('brand_cat')] = df[get_name_feat('brand')].factorize()[0]
df[get_name_feat('color_cat')] = df[get_name_feat('color')].factorize()[0]
df[get_name_feat('gender_cat')] = df[get_name_feat('gender')].factorize()[0]
df[get_name_feat('manufacturer part number_cat')] = df[get_name_feat('manufacturer part number')].factorize()[0]
df[get_name_feat('material')] = df[get_name_feat('material')].factorize()[0]
df[get_name_feat('sport_cat')] = df[get_name_feat('sport')].factorize()[0]
df[get_name_feat('style_cat')] = df[get_name_feat('style')].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]



In [40]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8846, 530)

In [78]:
feats_cat = [x for x in df.columns if '_cat' in x]
feats_cat

['brand_cat',
 'feat_catalog',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_item spec attr - style (ebay)_cat',
 'feat_case thickness_cat',
 'feat_case finish_cat',
 'feat_fastener_cat',
 'feat_profession_cat',
 'feat_package_cat',
 'feat_nike product #_cat',
 'feat_polarized?_cat',
 'feat_case material_cat',
 'feat_item package quantity_cat',
 'feat_recommended location_cat',
 'feat_lens material:_cat',
 'feat_diamond clarity_cat',
 'feat_lens color_cat',
 'feat_finish_cat',
 'feat_shipping to usa_cat',
 'feat_weather resistant_cat',
 'feat_is portable_cat',
 'feat_has mercury_cat',
 'feat_country of origin components:_cat',
 'feat_age range_cat',
 'feat_golf shoe type_cat',
 'feat_is dark sky-compliant_cat',
 'feat_материал_cat',
 'feat_fits styles_cat',
 'feat_sock style_cat',
 'feat_shoe closure_cat',
 'feat_is weather-resistant_cat',
 'feat_guaranteed authentic_cat

In [0]:
feats = ['brand_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)


In [120]:
feats = ['brand_cat', get_name_feat('brand_cat'), get_name_feat('metal type_cat') ,get_name_feat('gender_cat'), get_name_feat('material_cat'), get_name_feat('style_cat'), 
         get_name_feat('shape_cat'), 'feat_shoe category_cat', 'feat_size_cat', 'feat_color_cat', 'feat_condition_cat', 'feat_casual & dress shoe style_cat']

# list(set(feats + feats_cat))
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)
result

(-57.374391282217516, 4.174843944964274)

In [45]:
df[get_name_feat('material_cat')] = df[get_name_feat('material')]
df[get_name_feat('material_cat')].head()

0   -1
1   -1
2   -1
3   -1
4   -1
Name: feat_material_cat, dtype: int64

In [121]:
X = df[feats].values
y= df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)


(-57.374391282217516, 4.174843944964274)


Weight,Feature
0.2385 ± 0.0085,brand_cat
0.1018 ± 0.0066,feat_material_cat
0.0261 ± 0.0012,feat_gender_cat
0.0165 ± 0.0010,feat_brand_cat
0.0134 ± 0.0009,feat_shoe category_cat
0.0121 ± 0.0009,feat_shape_cat
0.0088 ± 0.0010,feat_metal type_cat
0.0064 ± 0.0021,feat_color_cat
0.0043 ± 0.0010,feat_style_cat
0.0018 ± 0.0002,feat_casual & dress shoe style_cat


In [52]:
df['brand'].value_counts()

nike                 1777
puma                  609
ralph lauren          526
vans                  386
new balance           371
                     ... 
1883 by wolverine       1
jyinstyle               1
carroll leather         1
dunham                  1
zorrie                  1
Name: brand, Length: 1732, dtype: int64

In [55]:
df[df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [122]:
!pwd


/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
!git add 